In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv('../TextFiles/smsspamcollection.tsv', sep='\t')

In [5]:
df.head()

,label,message,length,punct
0,ham,"Go until jurong point, crazy.. Available only ...",111,9
1,ham,Ok lar... Joking wif u oni...,29,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155,6
3,ham,U dun say so early hor... U c already then say...,49,6
4,ham,"Nah I don't think he goes to usf, he lives aro...",61,2


In [7]:
df.isnull().sum()

label      0
message    0
length     0
punct      0
dtype: int64

In [8]:
df['label'].value_counts()

ham     4825
spam     747
Name: label, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
X = df['message']
y = df['label']

In [12]:
X_train, X_test, y_train, y_test =  train_test_split(X, y, random_state=42, test_size=0.33)

In [13]:
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
count_vect = CountVectorizer()

In [17]:
# 1st Way
#count_vect.fit(X_train)
#X_train_counts = count_vect.transform(X_train)

# 2nd Way
X_train_counts = count_vect.fit_transform(X_train)

In [18]:
X_train_counts

<3733x7082 sparse matrix of type '<class 'numpy.int64'>'
	with 49992 stored elements in Compressed Sparse Row format>

In [20]:
X_train.shape

(3733,)

In [22]:
X_train_counts.shape

(3733, 7082)

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer

In [24]:
tfidf_transformer = TfidfTransformer()

In [25]:
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [26]:
X_train_tfidf.shape

(3733, 7082)

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [29]:
tfidf_vect = TfidfVectorizer()

In [31]:
X_train_tfidf = tfidf_vect.fit_transform(X_train)

In [32]:
from sklearn.svm import LinearSVC

In [33]:
clf = LinearSVC()

In [34]:
clf.fit(X_train_tfidf, y_train)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0)

In [35]:
from sklearn.pipeline import Pipeline

In [37]:
text_clf = Pipeline([('tfidf', TfidfVectorizer()),
                    ('clf', LinearSVC())])

In [39]:
text_clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [40]:
y_pred = text_clf.predict(X_test)

In [41]:
from sklearn.metrics import classification_report, confusion_matrix

In [43]:
print(confusion_matrix(y_test, y_pred))

[[1586    7]
 [  12  234]]


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ham       0.99      1.00      0.99      1593
        spam       0.97      0.95      0.96       246

   micro avg       0.99      0.99      0.99      1839
   macro avg       0.98      0.97      0.98      1839
weighted avg       0.99      0.99      0.99      1839



In [45]:
from sklearn.metrics import accuracy_score

In [46]:
print(accuracy_score(y_test, y_pred))

0.989668297988037


In [52]:
df['label'].value_counts()[0]/len(df)

0.8659368269921034

In [57]:
text_clf.predict(["Congratulations you have selected as winner"])

array(['spam'], dtype=object)